## Build Functions for ELT

In [2]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 uninstall pandas pyarrow fireducks numpy -y
# !pip3 install -r requirements.txt

In [3]:

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [4]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [6]:
f.predictions('amd')

AMD 5m Interval Timestamp: 2024-11-29 06:03:44 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,down
kelly_1:2.5,0.174894,0.182857,0.17515,0.130362
prob_up,0.4229,0.420838,0.408565,0.25
prob_static,0.331863,0.340932,0.327217,0.25
prob_down,0.245237,0.23823,0.264219,0.5
precision,0.410638,0.416327,0.410822,0.37883
recall,0.381423,0.403162,0.405138,0.268775
f1,0.395492,0.409639,0.40796,0.314451
support,"[506.0, 508.0, 506.0]","[506.0, 508.0, 506.0]","[506.0, 508.0, 506.0]","[506.0, 508.0, 506.0]"


AMD 15m Interval Timestamp: 2024-11-29 06:03:52 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,down,static
kelly_1:2.5,0.098936,0.047337,0.186592,0.172444
prob_up,0.407911,0.412311,0.325984,0.25
prob_static,0.247156,0.301111,0.330371,0.375
prob_down,0.344933,0.286578,0.343644,0.375
precision,0.356383,0.319527,0.418994,0.408889
recall,0.39881,0.321429,0.443787,0.544379
f1,0.376404,0.320475,0.431034,0.467005
support,"[168.0, 169.0, 169.0]","[168.0, 169.0, 169.0]","[168.0, 169.0, 169.0]","[168.0, 169.0, 169.0]"


AMD 1h Interval Timestamp: 2024-11-29 06:03:56 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,up
kelly_1:2.5,0.038655,0.057504,0.060737,0.067292
prob_up,0.280313,0.346501,0.331239,0.375
prob_static,0.273773,0.305176,0.297324,0.25
prob_down,0.445913,0.348324,0.371437,0.375
precision,0.313325,0.326788,0.329098,0.33378
recall,0.348465,0.311081,0.345794,0.332
f1,0.329962,0.318741,0.33724,0.332888
support,"[750.0, 750.0, 749.0]","[750.0, 750.0, 749.0]","[750.0, 750.0, 749.0]","[750.0, 750.0, 749.0]"


AMD 1wk Interval Timestamp: 2024-11-29 06:04:14 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,down,down,static
kelly_1:2.5,0.032353,0.129268,0.041304,0.07757
prob_up,0.374812,0.300088,0.309836,0.375
prob_static,0.250839,0.298236,0.302892,0.375
prob_down,0.374349,0.401676,0.387272,0.25
precision,0.308824,0.378049,0.315217,0.341121
recall,0.276316,0.402597,0.376623,0.477124
f1,0.291667,0.389937,0.343195,0.39782
support,"[152.0, 153.0, 154.0]","[152.0, 153.0, 154.0]","[152.0, 153.0, 154.0]","[152.0, 153.0, 154.0]"


AMD 1mo Interval Timestamp: 2024-11-29 06:04:17 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.12973,0.236364,0.066667,-0.008
prob_up,0.775391,0.720862,0.685302,0.875
prob_static,0.03666,0.131982,0.138455,0.0
prob_down,0.18795,0.147156,0.176243,0.125
precision,0.378378,0.454545,0.333333,0.28
recall,0.451613,0.483871,0.322581,0.225806
f1,0.411765,0.46875,0.327869,0.25
support,"[31.0, 30.0, 30.0]","[31.0, 30.0, 30.0]","[31.0, 30.0, 30.0]","[31.0, 30.0, 30.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')